In [1]:
from pathlib import Path
from typing import Tuple

def find_project_root() -> Path:
    """Find project root by looking for a marker file."""
    current = Path().resolve()
    for parent in [current] + list(current.parents):
        if (parent / "pyproject.toml").exists():  # or setup.py, .git, etc.
            return parent
    raise FileNotFoundError("Could not find project root")

PROJECT_ROOT = find_project_root()

DATA_DIR = PROJECT_ROOT / "data"

instances_base = DATA_DIR / "instances"
cache_base = DATA_DIR / "instances" / "caches"

# Run 3D4L on a Single Instance

This example shows how to define a custom pipeline runner to execute automated pipeline synthesis for a single instance. We utilize the FoodmartLoader class from ware_ops_algos to translate the instance file into a domain object compatible with 3D4L.


In [2]:
from ware_ops_algos.domain_models import BaseWarehouseDomain
from ware_ops_algos.data_loaders import FoodmartLoader
from ware_ops_pipes.utils.experiment_utils import PipelineRunner, RankingEvaluatorDistance


class SimpleRunner(PipelineRunner):
    def __init__(self, instance_set_name: str, 
                 instances_dir: Path, 
                 cache_dir: Path, 
                 project_root: Path):
        super().__init__(instance_set_name, 
                         instances_dir, 
                         cache_dir, 
                         project_root)
        
        self.loader = FoodmartLoader(str(instances_dir), 
                                     str(cache_dir))
        self.ranker = RankingEvaluatorDistance
        
    def discover_instances(self) -> list[Tuple[str, list[Path]]]:
        pass
    
    def load_domain(self, instance_name: str, 
                    file_paths: list[Path]) -> BaseWarehouseDomain:
        return self.loader.load(file_paths[0].name, use_cache=True)

runner = SimpleRunner("FoodmartData", instances_base / "FoodmartData",
                        cache_base / "FoodmartData", PROJECT_ROOT)

runner.run_instance(instance_name="instances_d5_ord5_MAL.txt", file_paths=[Path("data/instances/FoodmartData/instances_d5_ord5_MAL.txt")])

Loaded 38 model cards

Processing: instances_d5_ord5_MAL.txt

Problem type filtering for 'OBRP':
  Accepted types: {'order_selection', 'batching', 'routing', 'OBRP', 'item_assignment'}
  Result: 32/38 algorithms match

  Checking: ClarkAndWrightNN
    ✓ Algorithm is feasible

  Checking: ClarkAndWrightRR
    ✗ layout: constraint violated - n_blocks=2 does not satisfy {'equals': 1}

  Checking: ClarkAndWrightSShape
    ✓ Algorithm is feasible

  Checking: ClosestDepotMaxSharedArticlesSeedBatching
    ✓ Algorithm is feasible

  Checking: ClosestDepotMinDistanceSeedBatching
    ✓ Algorithm is feasible

  Checking: CombinedBatchingRoutingAssigning
    ✓ Algorithm is feasible

  Checking: DueDate
    ✗ orders: constraint violated - due_date=False does not satisfy {'equals': True}

  Checking: FiFo
    ✗ orders: constraint violated - order_date=False does not satisfy {'equals': True}

  Checking: GreedyIA
    ✓ Algorithm is feasible

  Checking: NNItemAssignment
    ✗ storage: type 'dedicate

C:\Users\zm0714\Documents\Projekte\ware_ops_pipes\.venv\Lib\site-packages\luigi\parameter.py:296: UserWarning: Parameter "instance_set_name" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
C:\Users\zm0714\Documents\Projekte\ware_ops_pipes\.venv\Lib\site-packages\luigi\parameter.py:296: UserWarning: Parameter "instance_name" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
C:\Users\zm0714\Documents\Projekte\ware_ops_pipes\.venv\Lib\site-packages\luigi\parameter.py:296: UserWarning: Parameter "instance_path" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
C:\Users\zm0714\Documents\Projekte\ware_ops_pipes\.venv\Lib\site-packages\luigi\parameter.py:296: UserWarning: Parameter "domain_path" with value "

Build repository...
Building tree grammar and inhabiting pipelines...
Enumerating up to 42 pipelines...
✓ Found 42 valid pipelines

Pipeline 1:
Task ResultAggregationDistance(config_index=, routing_plan={"__type__": "cls_luigi.inhabitation_task.RepoMeta.WrappedTask", "module": "ware_ops_pipes.pipelines.components.routing.pl", "task_class": "PLRouting", "arguments": [{"instance": {"__type__": "cls_luigi.inhabitation_task.RepoMeta.WrappedTask", "module": "ware_ops_pipes.pipelines.templates.template_1", "task_class": "InstanceLoader", "arguments": []}}, {"pick_list_plan": {"__type__": "cls_luigi.inhabitation_task.RepoMeta.WrappedTask", "module": "ware_ops_pipes.pipelines.components.batching.clark_and_wright_nn", "task_class": "ClarkAndWrightNN", "arguments": [{"instance": {"__type__": "cls_luigi.inhabitation_task.RepoMeta.WrappedTask", "module": "ware_ops_pipes.pipelines.templates.template_1", "task_class": "InstanceLoader", "arguments": []}}, {"order_selection_plan": {"__type__": "cls_lu

As the ranker shows, the best pipeline is the combination of GreedyItemAssignment, Local Search Batching with a RandomBatching component for constructing the initial solution and NearestNeighbourhoodRouting as the routing component